---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. Use this dataset to answer the questions below.

In [ ]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [ ]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
#answer_zero() 

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [ ]:
one_df = df.copy()
one_df['country'] = one_df.index
one_df = one_df.set_index('Gold')
one_df = one_df.reset_index()

def answer_one():
    most_summer_gold = max(one_df['Gold'])
    gold_df = one_df.where(one_df['Gold'] == most_summer_gold)
    gold_df = gold_df.dropna()
    #country = gold_mydf['country']
    return gold_df['country'].iloc[0]
answer_one()

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [ ]:
two_df = df.copy()
two_df['country'] = two_df.index
two_df = two_df.set_index('Gold')
two_df = two_df.reset_index()

def answer_two():
    summer_gold = two_df['Gold']
    winter_gold = two_df['Gold.1']
    abs_diff = abs(summer_gold - winter_gold)
    two_df['diff'] = abs_diff
    max_abs = max(two_df['diff'])
    max_abs
    max_abs_df = two_df.where(two_df['diff'] == max_abs)
    max_abs_df = max_abs_df.dropna()
    max_abs_df
    return max_abs_df['country'].iloc[0]    
answer_two()

### Question 3
Which country has the biggest difference between their summer and winter gold medal counts relative to their total gold medal count? Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [ ]:
three_df = df.copy()
three_df = three_df.where(three_df['Gold'] > 0)
three_df = three_df.dropna()

def answer_three():  
    for i, j in three_df.iterrows():
        summ = three_df.loc[i, 'Gold.2'] - three_df.loc[i, 'Gold']
        wint = three_df.loc[i, 'Gold.2'] - three_df.loc[i, 'Gold.1']
        three_df.loc[i, 'summ'] = summ
        three_df.loc[i, 'wint'] = wint
        
    max_diff = max(max(three_df['summ']), max(three_df['wint']))
    check = (three_df['summ'] == max_diff) | (three_df['wint'] == max_diff)
    j = three_df.where(check == True).dropna()
    return j.index[0]

### Question 4
Write a function to update the dataframe to include a new column called "Points" which is a weighted value where each gold medal counts for 3 points, silver medals for 2 points, and bronze mdeals for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [ ]:
four_df = df.copy()

def answer_four():    
    for i, j in four_df.iterrows():
        point = (3 * four_df.loc[i, 'Gold.2']) + (2 * four_df.loc[i, 'Silver.2']) + (1 * four_df.loc[i, 'Bronze.2'])
        four_df.loc[i, 'Points'] = point
    return four_df['Points'].astype(int)
answer_four()

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [ ]:
import pandas as pd
from operator import itemgetter 
census_df = pd.read_csv('census.csv')
census_df.head()

In [ ]:
five_df = census_df.copy()
columns_to_keep = ['STNAME',
                   'CTYNAME',
                   'STATE',
                   'DIVISION',
                   'COUNTY',
                   'SUMLEV',
                   'REGION',
                   'POPESTIMATE2010',
                   'POPESTIMATE2011',
                   'POPESTIMATE2012',
                   'POPESTIMATE2013',
                   'POPESTIMATE2014',
                   'POPESTIMATE2015']
five_df = five_df[columns_to_keep]

five_df = five_df[five_df['SUMLEV'] == 50]
five_df = five_df.set_index(['STNAME', 'CTYNAME'])


def answer_five():
    k = ((five_df.loc[state]['STATE'].count(), state) for state in five_df.index.levels[0])
    print(type(k))
    sorted_k = sorted(k, key = itemgetter(0), reverse=True)
    return sorted_k[0][1]

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)?

*This function should return a list of string values.*

In [ ]:
six_df = census_df.copy()
columns_to_keep = ['STNAME',
                   'CTYNAME',
                   'STATE',
                   'DIVISION',
                   'COUNTY',
                   'SUMLEV',
                   'REGION',
                   'POPESTIMATE2010',
                   'POPESTIMATE2011',
                   'POPESTIMATE2012',
                   'POPESTIMATE2013',
                   'POPESTIMATE2014',
                   'POPESTIMATE2015']
six_df = six_df[columns_to_keep]
six_df = six_df[six_df['SUMLEV'] == 50]
six_df = six_df.set_index(['STNAME', 'CTYNAME'])

def answer_six():
    s = ((state, sum_3_most_pop(state)) for state in six_df.index.levels[0])
    sorted_s = sorted(s, key = itemgetter(1), reverse=True)
    return [j[0] for j in sorted_s[:3]]

def sum_3_most_pop(state):
    sorted_state = six_df.loc[state].sort(columns='POPESTIMATE2015', axis = 0, ascending=False)
    return sum(sorted_state['POPESTIMATE2015'][:3])

### Question 7
Which county has had the largest change in population within the five year period (hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all five columns)?

*This function should return a single string value.*

In [ ]:
seven_df = census_df.copy()
seven_df = seven_df[seven_df['SUMLEV'] == 50]
col_to_keep = ['CTYNAME', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012',
               'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']
seven_df = seven_df[col_to_keep]
seven_df['10-11'] = abs(seven_df['POPESTIMATE2010'] - seven_df['POPESTIMATE2011'])
seven_df['11-12'] = abs(seven_df['POPESTIMATE2011'] - seven_df['POPESTIMATE2012'])
seven_df['12-13'] = abs(seven_df['POPESTIMATE2012'] - seven_df['POPESTIMATE2013'])
seven_df['13-14'] = abs(seven_df['POPESTIMATE2013'] - seven_df['POPESTIMATE2014'])
seven_df['14-15'] = abs(seven_df['POPESTIMATE2014'] - seven_df['POPESTIMATE2015'])

def answer_seven():
    max_pop_change = max(max(seven_df['10-11']), max(seven_df['11-12']), max(seven_df['12-13']), 
                     max(seven_df['13-14']), max(seven_df['14-15']))
    
    check = (seven_df['10-11']==max_pop_change) | (seven_df['11-12']==max_pop_change) | (seven_df['12-13']==max_pop_change) | \
            (seven_df['13-14']==max_pop_change)| (seven_df['14-15']==max_pop_change)

    j = seven_df.where(check == True).dropna()
    return j['CTYNAME'].iloc[0]

In [ ]:
#seven_df.where(seven_df['10-11'] == 95115).dropna()
#k = seven_df[seven_df['10-11'] == 95115].index.tolist()
#seven_df.loc[2667]['CTYNAME']

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [ ]:
eight_df = census_df.copy()
eight_df.head()

def answer_eight():
    eight_df = census_df.copy()
    eight_df = eight_df.where((eight_df['REGION'] == 1) | (eight_df['REGION'] == 2))
    eight_df = eight_df.where(eight_df['POPESTIMATE2015'] > eight_df['POPESTIMATE2014'])
    eight_df  = eight_df.where(eight_df['CTYNAME'].str.startswith('Washington'))
    columns_to_keep = ['STNAME', 'CTYNAME']
    eight_df = eight_df[columns_to_keep]
    return eight_df.dropna()